# Vigenère Cipher Implementation and Cryptanalysis
## Gabe DiMartino

This notebook implements a Vigenère cipher with an extended alphabet (A-Z plus space), demonstrates frequency analysis-based cryptanalysis using chi-squared statistics, and documents the process of breaking a 3-character key cipher. To run the full system please ensure that utils.py is also available.

## Setup and Constants

In [10]:
!pip install requests -q

In [11]:
from collections import Counter
import itertools
import re

from utils import add_space_and_rebalance, download_dictionary, search_google_books

# Define the alphabet including space character
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ "
CHAR_TO_NUM = {char: i for i, char in enumerate(ALPHABET)}
NUM_TO_CHAR = list(ALPHABET)
ALPHABET_SIZE = len(ALPHABET)

# Expected English letter frequencies from Cornell Mathematics
# Source: https://pi.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html
# Space frequency (absent from the Cornell data)  is estimated to be arount double that of 'E', 
# which altered the original distribution.

CORNELL_ORIGINAL = {
    'E': 0.1270, 'T': 0.0906, 'A': 0.0817, 'O': 0.0751, 'I': 0.0697,
    'N': 0.0675, 'S': 0.0633, 'H': 0.0609, 'R': 0.0599, 'D': 0.0425,
    'L': 0.0403, 'C': 0.0278, 'U': 0.0276, 'M': 0.0241, 'W': 0.0236,
    'F': 0.0223, 'G': 0.0202, 'Y': 0.0197, 'P': 0.0193, 'B': 0.0149,
    'V': 0.0098, 'K': 0.0077, 'J': 0.0015, 'X': 0.0015, 'Q': 0.0010, 
    'Z': 0.0007
}

EXPECTED_FREQ = add_space_and_rebalance(CORNELL_ORIGINAL)

# Convert to vector for chi-squared calculation
EXPECTED_VEC = [EXPECTED_FREQ.get(ch, 0.0) for ch in ALPHABET]

print(f"Alphabet size: {ALPHABET_SIZE}")
print(f"Expected frequency of 'E': {EXPECTED_FREQ['E']:.3f}")
print(f"Expected frequency of space: {EXPECTED_FREQ[' ']:.3f}")

Total frequency sum: 1.0000000000
Alphabet size: 27
Expected frequency of 'E': 0.095
Expected frequency of space: 0.254


## Part 1: Vigenère Cipher Functions

### Encryption and Decryption

In [12]:
def vigenere_encrypt(plaintext, key):
    """Encrypt plaintext using Vigenère cipher with given key."""
    key_nums = [CHAR_TO_NUM[ch] for ch in key]
    cipher_nums = []
    
    plain_nums = [CHAR_TO_NUM[ch] for ch in plaintext if ch in CHAR_TO_NUM]
    
    for i, p in enumerate(plain_nums):
        k = key_nums[i % len(key_nums)]
        cipher_nums.append((p + k) % ALPHABET_SIZE)
    
    return ''.join(NUM_TO_CHAR[n] for n in cipher_nums)

def vigenere_decrypt(ciphertext, key):
    """Decrypt ciphertext using Vigenère cipher with given key."""
    key_nums = [CHAR_TO_NUM[ch] for ch in key]
    plain_nums = []
    
    cipher_nums = [CHAR_TO_NUM[ch] for ch in ciphertext if ch in CHAR_TO_NUM]
    
    for i, c in enumerate(cipher_nums):
        k = key_nums[i % len(key_nums)]
        plain_nums.append((c - k) % ALPHABET_SIZE)
    
    return ''.join(NUM_TO_CHAR[n] for n in plain_nums)

# Test the implementation
test_plain = "HELLO WORLD"
test_key = "KEY"
encrypted = vigenere_encrypt(test_plain, test_key)
decrypted = vigenere_decrypt(encrypted, test_key)

print(f"Original:  '{test_plain}'")
print(f"Key:       '{test_key}'")
print(f"Encrypted: '{encrypted}'")
print(f"Decrypted: '{decrypted}'")

Original:  'HELLO WORLD'
Key:       'KEY'
Encrypted: 'RIIVSXFSOVH'
Decrypted: 'HELLO WORLD'


## Part 2: Approach to Breaking the Three-Character Key Vigenère Cipher

### Overview

My approach to breaking this Vigenère cipher leverages the fact that a three-character key essentially creates three independent Caesar ciphers operating on different positions of the text. I will begin by splitting the ciphertext into three columns, where column 1 contains characters at positions 0, 3, 6, 9, and so on; column 2 contains characters at positions 1, 4, 7, 10; and column 3 contains characters at positions 2, 5, 8, 11. Since each column represents text encrypted with a single Caesar shift, I can perform frequency analysis on each column independently.

### Frequency Analysis Strategy

For each column, I will test all 27 possible shift values and calculate how closely the resulting letter frequency distribution matches expected English letter frequencies using the chi-squared statistic. I'm using an adjusted Cornell Mathematics frequency table as my reference for standard English letter distributions, which provides reliable percentages for each letter's occurrence in typical English text. 

The chi-squared statistic is calculated using the formula:

**χ² = Σ [(O_i - E_i)² / E_i]**

Where:
- **O_i** is the observed frequency of character i in the decrypted text
- **E_i** is the expected frequency of character i based on standard English
- The sum is taken over all 27 characters (A-Z plus space)

This formula measures the "distance" between what you observe and what you expect. For each character, you calculate how far off our observed count is from the expected count, square this difference (to make all values positive and emphasize larger deviations), and divide by the expected count (to normalize the contribution). A perfect match would yield χ² = 0, while poor matches produce larger values. For example, if you decrypt with the wrong shift and get "QXVZ" where you expect common letters like "ETAO", the chi-squared value will be very high because you're seeing rare letters where common ones should be. When you hit the correct shift, common letters appear in their expected proportions, resulting in a low chi-squared value that signals you have found the right key character.

### Key Selection Process

Next, I will keep the top 5 candidates for each position based on their chi-squared scores. This gives me flexibility in case the text has unusual characteristics or is from older literature with slightly different frequency patterns. With 5 candidates for each of the 3 positions, I have 125 possible key combinations to test (5 × 5 × 5).

### Validation Through Dictionary Matching

To determine which of these 125 possible keys is correct, I will decrypt the ciphertext with each key combination and evaluate the results using a dictionary-based validation approach. For each decrypted text, I will count how many valid English words appear in the result. The key that produces the highest percentage of recognized English words is most likely to be correct. This dictionary validation step is crucial because it moves beyond pure statistical analysis to semantic validation. This is because real English text should produce real English words.

### Final Identification

Once I've identified the most likely key and decrypted the text, I will take a representative passage from the decrypted text and search for it online to identify the source work and author. This final step confirms that the decryption is correct and provides the complete answer to the problem. Throughout this process, I will maintain detailed logs of chi-squared scores, word match percentages, and any anomalies or interesting patterns I observe, as these notes will help debug any issues and understand why certain keys perform better than others.

### TLDR
1. **Column Separation**: Split the ciphertext into N columns where N is the key length
2. **Frequency Analysis**: Each column represents a Caesar cipher with a fixed shift
3. **Chi-Squared Testing**: Score each possible shift (0-26) against expected English frequencies
4. **Candidate Selection**: Keep top K candidates for each column position
5. **Combination Testing**: Try all combinations of top candidates
6. **Validation**: Check resulting plaintexts for English readability

## Frequency Analysis Functions
### Chi-Squared Scoring

In [13]:
def shift_decrypt_column(col_nums, shift):
    """Decrypt one column of the Vigenère cipher."""
    return [(c - shift) % ALPHABET_SIZE for c in col_nums]

def chi_square_score(col_nums, shift):
    """
    Compute chi-squared score for a given shift on one column.
    Lower scores indicate better match to expected English frequency.
    """
    decrypted = shift_decrypt_column(col_nums, shift)
    counts = Counter(decrypted)
    N = len(col_nums)
    chi2 = 0.0
    
    for i in range(ALPHABET_SIZE):
        observed = counts.get(i, 0)
        expected = EXPECTED_VEC[i] * N
        if expected > 0:
            chi2 += (observed - expected) ** 2 / expected
    
    return chi2

# Example: Test chi-squared scoring on a sample column
sample_text = "THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG"
sample_nums = [CHAR_TO_NUM[ch] for ch in sample_text if ch in CHAR_TO_NUM]

# Encrypt with shift of 3 and test scoring
shifted = [(n + 3) % ALPHABET_SIZE for n in sample_nums]

print("Chi-squared scores for different shifts:")
for shift in [0, 1, 2, 3, 4, 5]:
    score = chi_square_score(shifted, shift)
    print(f"  Shift {shift}: {score:.2f}")
print("\n(Lower score = better match to English)")

Chi-squared scores for different shifts:
  Shift 0: 264.45
  Shift 1: 788.78
  Shift 2: 175.25
  Shift 3: 121.52
  Shift 4: 3019.54
  Shift 5: 240.20

(Lower score = better match to English)


## Part 3: Complete Cryptanalysis Implementation

In [14]:
def crack_vigenere(ciphertext, key_length=3, top_k=5, max_output=10):
    """
    Crack a Vigenère cipher using frequency analysis.
    
    Parameters:
    - ciphertext: The encrypted text
    - key_length: Known or estimated key length (3 for this problem)
    - top_k: Number of top candidates to keep per column
    - max_output: Maximum number of results to return
    """
    # Convert ciphertext to numbers
    cipher_nums = [CHAR_TO_NUM[ch] for ch in ciphertext if ch in CHAR_TO_NUM]
    
    # Split into columns based on key length
    columns = [[] for _ in range(key_length)]
    for i, c in enumerate(cipher_nums):
        columns[i % key_length].append(c)
    
    print(f"Analyzing {len(cipher_nums)} characters in {key_length} columns")
    print(f"Column sizes: {[len(col) for col in columns]}\n")
    
    # Find best shifts for each column
    top_shifts = []
    for col_idx, col in enumerate(columns):
        scores = [(chi_square_score(col, s), s) for s in range(ALPHABET_SIZE)]
        best = sorted(scores, key=lambda x: x[0])[:top_k]
        
        print(f"Column {col_idx} top shifts (chi-squared scores):")
        for score, shift in best[:3]:
            key_char = NUM_TO_CHAR[shift]
            print(f"  {key_char:5} (shift {shift:2}): {score:.2f}")
        
        top_shifts.append([s for _, s in best])
    
    # Try all combinations of top shifts
    print(f"\nTrying {top_k**key_length} key combinations...")
    
    results = []
    for combo in itertools.product(*top_shifts):
        key = ''.join(NUM_TO_CHAR[s] for s in combo)
        plain = vigenere_decrypt(ciphertext, key)
        
        # Calculate combined chi-squared score
        total_chi = sum(chi_square_score(columns[i], combo[i]) 
                       for i in range(key_length))
        
        results.append((total_chi, key, plain))
    
    # Sort by score and return top results
    results.sort(key=lambda x: x[0])
    
    return results[:max_output]

def validate_with_dictionary(results, dictionary_words, min_word_length=3, show_details=True):
    """
    Validate decryption results using dictionary matching.
    
    Parameters:
    - results: List of (chi_score, key, plaintext) tuples from crack_vigenere
    - dictionary_words: Set of valid English words (uppercase)
    - min_word_length: Minimum word length to consider
    - show_details: Whether to print detailed results
    
    Returns:
    - List of tuples: (english_score, word_count, key, plaintext, sample_words)
    """
    validated_results = []
    
    if show_details:
        print("\nDictionary Validation Results")
        print("=" * 70)
    
    for chi_score, key, plaintext in results:
        # Extract potential words (consecutive letters)
        word_pattern = r'[A-Z]+' 
        potential_words = re.findall(word_pattern, plaintext)
        
        # Count valid words and characters
        valid_word_count = 0
        valid_char_count = 0
        total_char_count = 0
        found_words = []
        
        for word in potential_words:
            total_char_count += len(word)
            
            # Check if it's a valid word
            if len(word) >= min_word_length and word in dictionary_words:
                valid_word_count += 1
                valid_char_count += len(word)
                found_words.append(word)
            elif len(word) == 1:  # Single letters like 'I' or 'A'
                if word in ['A', 'I']:
                    valid_word_count += 1
                    valid_char_count += 1
                    found_words.append(word)
        
        # Calculate English score (percentage of valid characters)
        english_score = valid_char_count / total_char_count if total_char_count > 0 else 0
        
        # Store results
        validated_results.append((
            english_score,
            valid_word_count,
            chi_score,
            key,
            plaintext,
            found_words[:10]  # Keep first 10 words as sample
        ))
    
    # Sort by English score (descending)
    validated_results.sort(key=lambda x: x[0], reverse=True)
    
    if show_details:
        # Display top 3 results
        for i, (eng_score, word_count, chi_score, key, plaintext, sample_words) in enumerate(validated_results[:3]):
            print(f"\nCandidate {i+1}:")
            print(f"  Key: '{key}'")
            print(f"  English Score: {eng_score:.1%} ({word_count} valid words)")
            print(f"  Chi-squared: {chi_score:.2f}")
            print(f"  Sample words: {', '.join(sample_words[:5])}")
            print(f"  First 100 chars: {plaintext[:100]}...")
    
    return validated_results

### Ciphertext and Parameter Configuration

In [15]:
# Configuration Parameters
CIPHERTEXT = """
RTMRSMJ F UFQ GWIHNREJLYG NLK XWYNDTNDMBHNB NISVWGYN ZJNMRNMYOEAEEAIAW AO NIGAXDMOODBU N ZCSFIBRII IWNWTMXFMJ IRFRIHAEEHNRMUIFBLRIK XW ITUN SNEYRNTA A  HREIA AO ECCUIAMVA IMNG ON AW URSMOIDATMNNFNRVWGMJ  NITQBACRUXOQITURSMBRGBHMRSMAOMEEYU SRXRM VW FQEMVI MSMXFMBHRISG RACNQRNTIFNVIYREEITUJTMQEMRSMLO AIQNRRM NA FQEM ITQTSCLMYRAYEDBYMXFMAOZN AWEMXRMXTUNRMXFMBHRRRMMAGPHFNREIMKIDRJRMVRMKE WEFISNRDMQIEILNMYMBOMQIZIO N QJYMQAHN KXUMQEN DMBHNB  NTUNRSREYM BJRXIIEILRB NB YJSFIMDIBRWNRB DNPYREQITUJTMQEMQAQINAB OCTMRTMRSM EFCR NDMAHRIFA  Z SMUO P UJSMSUEB ONE IHR EMJNQISUN FXLQIMRIAYU NKOGB VB Z  ONN NTMVAQN  X NWSINRMMOMWOFIYAC IJNFITAIK XWMEHAIHNA FJKRW VB P IRM URSMEISN VVPNBIRWTYG KXUMEA B FX FNLYIMRIA M VIHNDEMWOMXBWNCFRO ITAIHRJRVWGMRT
"""

# Analysis parameters
KEY_LENGTH = 3         # Known key length (3 characters)
TOP_K = 5              # Number of top candidates per column (5 means 125 total combinations)
SHOW_TOP_RESULTS = 3   # Number of best results to display

print(f"\nConfiguration:")
print(f"  Key length: {KEY_LENGTH}")
print(f"  Top candidates per column: {TOP_K}")
print(f"  Total key combinations to test: {TOP_K**KEY_LENGTH}")

print(f"\nCiphertext Statistics:")
print(f"  Original length: {len(CIPHERTEXT)} characters")
print(f"  First 100 chars: {CIPHERTEXT[:100]}...")

print("\n" + "-" * 80)
print("LOADING DICTIONARY")
print("-" * 80)

dictionary_words = download_dictionary(max_words=370000)

print(f"Dictionary loaded: {len(dictionary_words)} words")


Configuration:
  Key length: 3
  Top candidates per column: 5
  Total key combinations to test: 125

Ciphertext Statistics:
  Original length: 742 characters
  First 100 chars: 
RTMRSMJ F UFQ GWIHNREJLYG NLK XWYNDTNDMBHNB NISVWGYN ZJNMRNMYOEAEEAIAW AO NIGAXDMOODBU N ZCSFIBRII ...

--------------------------------------------------------------------------------
LOADING DICTIONARY
--------------------------------------------------------------------------------
Loading cached dictionary from english_dictionary.txt...
Loaded 370000 words from cache
Dictionary loaded: 370000 words


### Cryptanalysis for Vigenère using Chi-Squared Frequency Analysis

In [16]:
# Crack the cipher using frequency analysis
chi_results = crack_vigenere(
    CIPHERTEXT, 
    key_length=KEY_LENGTH, 
    top_k=TOP_K, 
    max_output=TOP_K**KEY_LENGTH  # Get all combinations
)

print(f"\nGenerated {len(chi_results)} possible decryptions")

# Display top chi-squared results
print("\nTop 5 results by chi-squared score:")
print("-" * 40)
for i, (chi_score, key, plaintext) in enumerate(chi_results[:5], 1):
    print(f"{i}. Key: '{key}' | Chi²: {chi_score:.2f} | Start: {plaintext[:30]}...")


Analyzing 740 characters in 3 columns
Column sizes: [247, 247, 246]

Column 0 top shifts (chi-squared scores):
  J     (shift  9): 32.99
  W     (shift 22): 713.63
  Q     (shift 16): 1094.03
Column 1 top shifts (chi-squared scores):
  A     (shift  0): 28.70
  N     (shift 13): 914.04
  H     (shift  7): 1172.80
Column 2 top shifts (chi-squared scores):
  N     (shift 13): 29.70
  M     (shift 12): 816.81
  Z     (shift 25): 851.46

Trying 125 key combinations...

Generated 125 possible decryptions

Top 5 results by chi-squared score:
----------------------------------------
1. Key: 'JAN' | Chi²: 91.40 | Start: IT IS A TRUTH UNIVERSALLY ACKN...
2. Key: 'WAN' | Chi²: 772.03 | Start: WT WS O TEUTV UAIVSRSOLLL AQKN...
3. Key: 'JAM' | Chi²: 878.50 | Start: ITAISAA URUUH VNIWERTALMY BCKO...
4. Key: 'JAZ' | Chi²: 913.15 | Start: ITOISOA HRUHH INIJERGAL Y PCKB...
5. Key: 'JNN' | Chi²: 976.74 | Start: IG IF ANTRHTHNUNWVEESAZLYNACYN...


### Compare Top results with English Dictionary

In [17]:
# Validate all results with dictionary
validated_results = validate_with_dictionary(
    chi_results, 
    dictionary_words,
    min_word_length=3,
    show_details=False
)

# Display top validated results
print(f"\nTop {SHOW_TOP_RESULTS} results by English word matching:")
print("-" * 60)

best_key = None
best_plaintext = None
result = None

for i, (eng_score, word_count, chi_score, key, plaintext, sample_words) in enumerate(validated_results[:SHOW_TOP_RESULTS], 1):
    print(f"\n{i}. KEY: '{key}'")
    print(f"   English Score: {eng_score:.1%} ({word_count} valid words found)")
    print(f"   Chi-squared: {chi_score:.2f}")
    print(f"   Sample valid words: {', '.join(sample_words[:8])}")
    print(f"   Plaintext start: {plaintext[:150]}...")
    
    # Save the best result
    if i == 1:
        best_key = key
        best_plaintext = plaintext



Top 3 results by English word matching:
------------------------------------------------------------

1. KEY: 'JAN'
   English Score: 83.9% (107 valid words found)
   Chi-squared: 91.40
   Sample valid words: A, TRUTH, UNIVERSALLY, ACKNOWLEDGED, THAT, A, SINGLE, MAN
   Plaintext start: IT IS A TRUTH UNIVERSALLY ACKNOWLEDGED THAT A SINGLE MAN IN POSSESSION OF A GOOD FORTUNE MUST BE IN WANT OF A WIFE HOWEVER LITTLE KNOWN THE FEELINGS O...

2. KEY: 'CAN'
   English Score: 3.9% (10 valid words found)
   Chi-squared: 1805.28
   Sample valid words: MOR, UNL, SIT, TOE, TOE, A, BAT, I
   Plaintext start: PT PS H TYUTO UUIVLRSHLLE AJKNVWLLDGLD  HA  AGSIUGLL MHN PN WOSZESZIOU OM AGGOVD MOR UNL MASTGBEGINGWAUT VF H WPFEGHOCEVLR SIT LEGKNVWNGTHL FLELPNGZ O...

3. KEY: 'QAN'
   English Score: 3.7% (7 valid words found)
   Chi-squared: 1152.43
   Sample valid words: MUN, PELE, TAE, TAE, MKS, EON, FADY
   Plaintext start: BT BS U TKUTA UGIVYRSULLR AWKNHWLYDGYD MHAM ATSIGGLY MUN BN IOSLESLIOG OZ ATGO

### Search Google Books

In [18]:
if best_plaintext:    
    print(f"\nBest Key Found: '{best_key}'")
    print("\nDecrypted Text Excerpt (first 500 characters):")
    print("-" * 60)
    print(best_plaintext[:500])
    
    print("\n" + "-" * 60)
    print("IDENTIFYING SOURCE...")
    print("-" * 60)
        
    # Extract a clean passage for searching
    words = best_plaintext.split()
    # Skip fragmented beginning if needed
    start_idx = 0
    for i in range(min(10, len(words))):
        if i > 0 and len(words[i]) > 3:
            start_idx = i
            break
    
    # Create search query from a good passage
    search_text = ' '.join(words[start_idx:start_idx+30])  # About 30 words
    
    # Search Google Books
    result = search_google_books(search_text)
else:
    print("\nNo valid decryption found. Try adjusting parameters:")
    print("  - Increase TOP_K for more key combinations")
    print("  - Use a larger dictionary")
    print("  - Check if the ciphertext is correctly formatted")
    



Best Key Found: 'JAN'

Decrypted Text Excerpt (first 500 characters):
------------------------------------------------------------
IT IS A TRUTH UNIVERSALLY ACKNOWLEDGED THAT A SINGLE MAN IN POSSESSION OF A GOOD FORTUNE MUST BE IN WANT OF A WIFE HOWEVER LITTLE KNOWN THE FEELINGS OR VIEWS OF SUCH A MAN MAY BE ON HIS FIRST ENTERING A NEIGHBOURHOOD THIS TRUTH IS SO WELL FIXED IN THE MINDS OF THE SURROUNDING FAMILIES THAT HE IS CONSIDERED AS THE RIGHTFUL PROPERTY OF SOME ONE OR OTHER OF THEIR DAUGHTERS MY DEAR MR BENNET SAID HIS LADY TO HIM ONE DAY HAVE YOU HEARD THAT NETHERFIELD PARK IS LET AT LAST MR BENNET REPLIED THAT HE HA

------------------------------------------------------------
IDENTIFYING SOURCE...
------------------------------------------------------------
Searching Google Books API...

✓ Found 3 matches in Google Books
Best match: 'Pride and Prejudice' by Jane Austen

Other possible matches:
  1. 'The Novels of Jane Austen: Pride and prejudice' by Jane Austen
  2. 'A Truth 

### Results

In [19]:
if result:
    print(f"\n1. KEY: '{best_key}'")
    
    if result and result.get('found'):
        print(f"2. WORK: {result['title']}")
        print(f"3. AUTHOR: {result['author']}")
        
        if result.get('published'):
            print(f"\nAdditional Info:")
            print(f"  Published: {result['published']}")
            print(f"  Confidence: {result['confidence']}%")
        
        print(f"\nSuccessfully identified the source text")
    else:
        # Fallback if API search fails
        print(f"2. WORK: [Could not auto-identify]")
        print(f"3. AUTHOR: [Could not auto-identify]")
        
        print(f"\nSearch manually using this text:")
        print(f'"{search_text[:150]}..."')
        
        # Provide manual search URLs as backup
        from urllib.parse import quote
        encoded = quote(search_text[:100])
        print(f"\nManual search links:")
        print(f"  • https://www.google.com/search?q={encoded}")
        print(f"  • https://books.google.com/books?q={encoded}")
else:
    print("\nNo valid decryption found. Try adjusting parameters:")
    print("  - Increase TOP_K for more key combinations")
    print("  - Use a larger dictionary")
    print("  - Check if the ciphertext is correctly formatted")
    



1. KEY: 'JAN'
2. WORK: Pride and Prejudice
3. AUTHOR: Jane Austen

Additional Info:
  Published: 1853
  Confidence: 90%

Successfully identified the source text


As revealed by the analysis the Key is JAN, the work is Pride and Prejudice and the author is Jane Austen.